# LightGCN

## 1. Data preprocessing

### 1.1. Loading and splitting the data

### 1.2. Preprocess data

## 2. Create and train model

## 3. Evaluate

### 3.1. Generate recommendations

### 3.2. Evaluate using NDCG@20

## 4. Tune model parameters

## 5. Recommend

### 5.1. Train model with best params and on full dataset

### 5.2 Generate predictions file